In [ ]:
# this example is still really messy! I created it a while ago and have learnt more since.
# i will update when I have time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Pull in the data from kaggle
data = pd.read_csv("/kaggle/input/craigslist-carstrucks-data/vehicles.csv") 

In [ ]:
#Have an initial look at the data. I also used data browser.
data.describe()

In [ ]:
# Pull in some columns I think I can show a simple relationship on. I will then build a simple model for a baseline and establish the RMSE. Then I will build a second
# Improved model using polynomial regression. I think the question here is, what price is the BMW based on the milage alone?

selected_columns = data[["id","price","odometer","manufacturer"]]

#Index into desired columns to copy

selected_df = selected_columns.copy()

print("Are There Missing Data? :",selected_df.isnull().any().any())
# There is but it still seems to work. Will investigate later.

print(selected_df)

In [ ]:
#Have a look at the first five records. It looks straightforward.
selected_df.head()

In [ ]:
#Must check the data for outliers
sns.scatterplot(data = selected_df, x = "price", y = "odometer")

In [ ]:
# Deleting unwanted records. I looked at the distribution graph and saw some really high values for both Odometer and Price.
#I am going to delete them...
# cutoff at 300,000km and $40K

selected_df1 = selected_df[selected_df['odometer'] < 300000]
selected_df2 = selected_df1[selected_df1['odometer'] > 1000]
selected_df3 = selected_df2[selected_df2['price'] > 1000]
selected_df4 = selected_df3[selected_df3['price'] < 40000]
selected_df_clean = selected_df4[selected_df4['manufacturer'] == 'bmw']
# Interested to see if there is a correlation for one manufacturer vs all (ans: much clearer with one manufacturer)

In [ ]:
#Print the regression plot, with simple trend line. An improved model could be a polynominal regression.
sns.regplot(selected_df_clean['price'], selected_df_clean['odometer'], scatter_kws={"color": "black"}, order=1, line_kws={"color": "red"})

In [ ]:
#Print the regression plot, with simple trend line model is 2nd order.
sns.regplot(selected_df_clean['price'], selected_df_clean['odometer'],scatter_kws={"color": "black"}, order=2, line_kws={"color": "red"})

In [ ]:
#Print the regression plot, with simple trend line model is 3nd order.
sns.regplot(selected_df_clean['price'], selected_df_clean['odometer'],scatter_kws={"color": "black"}, order=3, line_kws={"color": "red"})

In [ ]:
# Now I will try to build a model based on what I have seen above

In [ ]:
# Try to add a feature [manufacturer]
x = selected_df_clean[['odometer']]
y = selected_df_clean[['price']]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.4, random_state= 101)

In [ ]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(x_train, y_train)

In [ ]:
x_train.head()


In [ ]:
y_train.head()

In [ ]:
coeff_df = pd.DataFrame(lm.coef_,x.columns,columns=['Coefficient'])
coeff_df

In [ ]:
predictions = lm.predict(x_test)
print(predictions)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test, predictions)

In [ ]:
# Plot outputs
plt.xlabel('Odometer')
plt.ylabel('Price')
plt.title('Price vs Odometer')
plt.grid(True)

plt.scatter(x_test, y_test,  color='black')
plt.plot(x_test, predictions, color='blue', linewidth=1)

plt.show()

In [ ]:
coeff_df

In [ ]:
print(lm.intercept_)

In [ ]:
from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
#Can we predict anything?
odoreading = [0],[100000]
lm.predict(odoreading)
#Apparently yes. 100,000 KM BMW = $14,757 ! Looks about right from the graph above.